In [2]:
import pandas as pd
import quandl
from datetime import datetime
from config import QUANDL_API_KEY
import yfinance as yf

In [3]:
# Set the API key
quandl.ApiConfig.api_key = QUANDL_API_KEY

In [4]:
def get_adjusted_close_prices(tickers, start_date, end_date=None):
    """
    Retrieves adjusted close price data for given tickers from Quandl.

    Parameters:
        tickers (list): A list of ticker symbols (e.g., ['SPY', 'LQD']).
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str, optional): The end date in 'YYYY-MM-DD' format. Defaults to today.

    Returns:
        pandas.DataFrame: A DataFrame containing adjusted close prices with dates as the index.
    """

    # Set end_date to today if not provided
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')

    # Prepare Quandl codes for the tickers
    quandl_codes = [f"EOD/{ticker}" for ticker in tickers]

    try:
        # Retrieve data for all tickers in one API call
        data = quandl.get(quandl_codes,
                          start_date=start_date,
                          end_date=end_date,
                          paginate=True)

        # Create an empty DataFrame for adjusted close prices
        adj_close_df = pd.DataFrame()

        # Extract 'Adj_Close' for each ticker
        for ticker in tickers:
            adj_close = data[f'EOD/{ticker}']['Adj_Close'].rename(ticker)
            adj_close_df = pd.concat([adj_close_df, adj_close], axis=1)

        # Sort the DataFrame by date
        adj_close_df.sort_index(inplace=True)

        return adj_close_df

    except Exception as e:
        print(f"Error retrieving data: {e}")
        return pd.DataFrame()

In [5]:
def yf_adj(tickers, start_date, end_date=None):
    """
    Retrieves adjusted close price data for given tickers from Yahoo Finance using yfinance.

    Parameters:
        tickers (list): A list of ticker symbols (e.g., ['SPY', 'LQD']).
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str, optional): The end date in 'YYYY-MM-DD' format. Defaults to today.

    Returns:
        pandas.DataFrame: A DataFrame containing adjusted close prices with dates as the index.
    """
    # Set end_date to today if not provided
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')

    try:
        # Retrieve data from Yahoo Finance
        data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']

        # Ensure data is a DataFrame even if only one ticker is provided
        if isinstance(data, pd.Series):
            data = data.to_frame()

        # Sort the DataFrame by date
        data.sort_index(inplace=True)

        # Change date format
        data.index = data.index.strftime('%Y-%m-%d')

        return data

    except Exception as e:
        print(f"Error retrieving data: {e}")
        return pd.DataFrame()

# Asset Classes:

We will retreive adjusted close prices for 7 ETFs to broadly represent the 7 main asset classes as defined by AQR Capital Management. The ETFs are as follows:

1.- **US Equities:**

*   Vanguard Total Stock Market ETF (VTI)
*   Description: Seeks to track the performance of the CRSP US Total Market Index, covering virtually the entire U.S. investable equity market, including small-, mid-, and large-cap stocks.

2.- **Non-US Developed Equities:**

Defined as a cap-weighted average of Euro-5, Japan, U.K., Australia, and Canada.

*   iShares MSCI EAFE ETF (EFA)
*   Description: Tracks the MSCI EAFE Index, focusing on large- and mid-cap stocks in developed markets outside North America, specifically Europe, Australasia, and the Far East.

3.- **Emerging Market Equities**

*   iShares MSCI Emerging Markets ETF (EEM)
*   Description: Tracks the MSCI Emerging Markets Index, providing exposure to large- and mid-cap companies in emerging market countries.

4.- **High Yield US Credit**
*   iShares iBoxx $ High Yield Corporate Bond ETF (HYG)
*   Description: Tracks the Markit iBoxx USD Liquid High Yield Index, focusing on U.S. dollar-denominated high-yield corporate bonds.


5.- **US Investment Grade Credit**

*   iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD)
*   Description: Tracks the Markit iBoxx USD Liquid Investment Grade Index, comprising U.S. dollar-denominated, investment-grade corporate bonds.

6.- **US 10-Year Treasuries**
*   iShares 7-10 Year Treasury Bond ETF (IEF)
*   Description: Tracks the ICE U.S. Treasury 7-10 Year Bond Index, providing exposure to U.S. Treasury bonds with remaining maturities between 7 and 10 years.

7.- **Non-US 10-Year Government Bonds**

Defined as a GDP-weighted average of Germany, Japan, U.K., Australia, and Canada.

*   iShares International Treasury Bond ETF (IGOV)
*   Description: Tracks the FTSE World Government Bond Index – Developed Markets Capped Select Index, offering exposure to fixed-rate, local currency sovereign debt of investment-grade countries outside the U.S., including the specified countries.


**Please note:** ETFs are a proxy of each asset class and are not built to perfectly represent the asset class.

In [132]:
asset_classes = ['VTI', 'EFA', 'EEM', 'HYG', 'LQD', 'IEF', 'IGOV']
df = yf_adj(asset_classes, '1995-01-01')

In [134]:
# Save dataframes to CSV in the data folder
df.to_csv(r'./data/adj_prices.csv')